In [1]:
# Use this command to install the local development version of AEON.py if desired
!/usr/bin/python3 -m pip install --force-reinstall ./../../../target/wheels/biodivine_aeon-1.0.0a10-cp37-abi3-macosx_11_0_arm64.whl

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for biodivine-aeon==1.1.0 from https://files.pythonhosted.org/packages/51/18/602f688f949248dc3c792a8e2e742d7d5449d7ffa89cf97de465578588de/biodivine_aeon-1.1.0-cp37-abi3-macosx_11_0_x86_64.macosx_11_0_arm64.macosx_11_0_universal2.whl.metadata
  Using cached biodivine_aeon-1.1.0-cp37-abi3-macosx_11_0_x86_64.macosx_11_0_arm64.macosx_11_0_universal2.whl.metadata (10 kB)
Using cached biodivine_aeon-1.1.0-cp37-abi3-macosx_11_0_x86_64.macosx_11_0_arm64.macosx_11_0_universal2.whl (5.6 MB)
  Attempting uninstall: biodivine-aeon
    Found existing installation: biodivine_aeon 1.1.0
    Uninstalling biodivine_aeon-1.1.0:
      Successfully uninstalled biodivine_aeon-1.1.0

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


## Demonstration of control on Myeloid model

The model we will observe is coming from Krumsiek, J., Marr, C., Schroeder, T., and Theis, F. J. (2011). Hierarchical differentiation of myeloid progenitors is encoded in the transcription factor network. PloS one, 6(8), e22649.

It describes myeloid cell differentiation into four possible types - erythrocytes, megakaryocytes, monocytes, and granulocytes.

![Myeloid_differentiation](myeloid.png)

In [2]:
import biodivine_aeon as ba 
import pandas as pd

In [3]:
ba.LOG_LEVEL = ba.LOG_NOTHING

In [4]:
# Load the model and create a Perturbation Graph

model = ba.BooleanNetwork.from_file('myeloid_witness.aeon')
pstg = ba.AsynchronousPerturbationGraph(model)

In [5]:
# Find attractors representing the four possible phenotypes

attractors = ba.Attractors.attractors(pstg)
attractors = [ a.vertices() for a in attractors ]

erythrocyte = pstg.mk_subspace_vertices({ "EKLF": True })
erythrocyte_att = [ a for a in attractors if not a.intersect(erythrocyte).is_empty() ][0]
print('erythrocyte attractor size (should be 1):', erythrocyte_att.cardinality())
print('erythrocyte attractor:', [sorted(x.to_named_dict().items()) for x in  erythrocyte_att.items()])

megakaryocyte = pstg.mk_subspace_vertices({ "Fli1": True })
megakaryocyte_att = [ a for a in attractors if not a.intersect(megakaryocyte).is_empty() ][0]
print('megakaryocyte attractor size (should be 1):', megakaryocyte_att.cardinality())
print('megakaryocyte megakaryocyte:', [sorted(x.to_named_dict().items()) for x in  megakaryocyte_att.items()])

monocyte = pstg.mk_subspace_vertices({ "cJun": True })
monocyte_att = [ a for a in attractors if not a.intersect(monocyte).is_empty() ][0]
print('monocyte attractor size (should be 1):', monocyte_att.cardinality())
print('monocyte attractor:', [sorted(x.to_named_dict().items()) for x in  monocyte_att.items()])


granulocyte = pstg.mk_subspace_vertices({ "Gfi1": True })
granulocyte_att = [ a for a in attractors if not a.intersect(granulocyte).is_empty() ][0]
print('granulocyte attractor size (should be 1):', granulocyte_att.cardinality())
print('granulocyte attractor:', [sorted(x.to_named_dict().items()) for x in  granulocyte_att.items()])

erythrocyte attractor size (should be 1): 1
erythrocyte attractor: [[('CEBPa', False), ('EKLF', True), ('EgrNab', False), ('FOG1', True), ('Fli1', False), ('GATA1', True), ('GATA2', False), ('Gfi1', False), ('PU1', False), ('SCL', True), ('cJun', False)]]
megakaryocyte attractor size (should be 1): 1
megakaryocyte megakaryocyte: [[('CEBPa', False), ('EKLF', False), ('EgrNab', False), ('FOG1', True), ('Fli1', True), ('GATA1', True), ('GATA2', False), ('Gfi1', False), ('PU1', False), ('SCL', True), ('cJun', False)]]
monocyte attractor size (should be 1): 1
monocyte attractor: [[('CEBPa', False), ('EKLF', False), ('EgrNab', True), ('FOG1', False), ('Fli1', False), ('GATA1', False), ('GATA2', False), ('Gfi1', False), ('PU1', True), ('SCL', False), ('cJun', True)]]
granulocyte attractor size (should be 1): 1
granulocyte attractor: [[('CEBPa', True), ('EKLF', False), ('EgrNab', False), ('FOG1', False), ('Fli1', False), ('GATA1', False), ('GATA2', False), ('Gfi1', True), ('PU1', True), ('SCL'

In [6]:
# Let's demonstrate now, how we can compute source-target control.

# First we need to compute a symbolic structure containing the perurbations and results
sym_results = ba.Control.attractor_one_step(pstg, erythrocyte_att, megakaryocyte_att)
sym_results

ColoredPerturbationSet(cardinality=5832, colors=1, perturbations=5832, symbolic_size=12)

In [7]:
# Now we can either look up for what colors does a specific perturbation work
sym_results.select_perturbation({ "EKLF" : True }) # A perturbation where only EKLF is perturbed to 1.
# Cardinality 0 -> not working

ColorSet(cardinality=0, symbolic_size=1)

In [8]:
sym_results.select_perturbation({ "EKLF" : False }) # A perturbation where only EKLF is perturbed to 0
# Cardinality 0 -> not working

ColorSet(cardinality=0, symbolic_size=1)

In [9]:
sym_results.select_perturbation({ "EKLF" : None }) # A perturbations where EKLF is unperturbed (and everything else is also unperturbed).
# Cardinality 0 -> not working

ColorSet(cardinality=0, symbolic_size=1)

In [10]:
sym_results.select_perturbation({ "EKLF" : False, "Fli1": True }) # A perturbation that we know works.

ColorSet(cardinality=1, symbolic_size=13)

In [11]:
# Or just retrieve all smallest colours with a given minimal robustness to avoid guessing
(p, r, c) = sym_results.select_by_robustness(threshold=0.99, result_limit=10)[0]
print(p.perturbed_named_dict())

{'EKLF': False, 'Fli1': True}


In [12]:
# In a similar manner, we can also compute phenotype control. Nonetheless, providing just the target state 
# (or a set of states) is sufficient in this case)

# There are three possible configurations of oscillation
# Forbidden -> attractor where model stabilizes therefore cannot oscillate through the phenotype (must stabilize fully within the phenotype)
# Allowed -> attractor where model stabilizes may oscillate
# Required -> attractor where model stabilizes MUST oscillate through the phenotype (i.e. have states both inside and outside of the phenotype space)

# In this demonstration we use forbidden type of oscillation, since myeloid model does not show oscillations 
# and this type is most similar to the source-target control.

# Even though the model is small, using ceiled version of the function is recommended, since it allows stop_early 
# functionality, computing the perturbations until 100% robustness is hit or specifying a sensible size_bounc
phen_sym_results = ba.Control.phenotype_permanent(
    graph=pstg,
    phenotype=megakaryocyte_att,
    oscillation_type="forbidden",
    size_limit=10,
    stop_when_found=True
)

In [13]:
(p, r, c) = phen_sym_results.select_by_robustness(threshold=0.99, result_limit=1)[0]
print(p.perturbed_named_dict())

{'PU1': False, 'Fli1': True}


In [14]:
# Now we can compute a full sets of results for all pairs of source-target control

In [15]:
all_atts = {
    "Erythrocyte": erythrocyte_att,
    "Megakaryocyte": megakaryocyte_att,
    "Monocyte": monocyte_att,
    "Granulocyte": granulocyte_att
}

# Generate al pair possibilities for computing the source-target control
all_att_pairs = [(s,t) for s in all_atts.keys() for t in all_atts.keys() if s != t]

In [16]:
# Pick the minimal perturbations with perfect robustness and then only return the
# values of variables that are actually perturbed (to simplify presentation).
# We are not returning the robustness here becasue it is always 1.0 anyway.
def pick_result(data: ba.ColoredPerturbationSet) -> dict[str, bool]:
    (min_model, _, _) = data.select_by_robustness(threshold=1.0, result_limit=1)[0]
    picked = data.select_by_size(size=min_model.perturbation_size(), up_to=False)
    picked = picked.select_by_robustness(threshold=1.0, result_limit=100)
    return [ model.perturbed_named_dict() for (model, _, _) in picked ]

data = {
    'source': [s for s, _ in all_att_pairs],
    'target': [t for _, t in all_att_pairs],
    'onestep': [pick_result(ba.Control.attractor_one_step(pstg, all_atts[s], all_atts[t])) for s, t in all_att_pairs],
    'temporary': [pick_result(ba.Control.attractor_temporary(pstg, all_atts[s], all_atts[t])) for s, t in all_att_pairs],
    'permanent': [pick_result(ba.Control.attractor_permanent(pstg, all_atts[s], all_atts[t])) for s, t in all_att_pairs],
}

df = pd.DataFrame(data)

In [17]:
df

,source,target,onestep,temporary,permanent
0,Erythrocyte,Megakaryocyte,"[{'Fli1': True, 'EKLF': False}]","[{'Fli1': True}, {'EKLF': False}]","[{'Fli1': True}, {'EKLF': False}]"
1,Erythrocyte,Monocyte,[{'PU1': True}],[{'PU1': True}],[{'PU1': True}]
2,Erythrocyte,Granulocyte,"[{'Gfi1': True, 'GATA1': False, 'CEBPa': True}]","[{'cJun': False, 'CEBPa': True}, {'CEBPa': Tru...","[{'CEBPa': True, 'cJun': False}, {'Gfi1': True..."
3,Megakaryocyte,Erythrocyte,"[{'Fli1': False, 'EKLF': True}]","[{'Fli1': False}, {'EKLF': True}]","[{'Fli1': False}, {'EKLF': True}]"
4,Megakaryocyte,Monocyte,[{'PU1': True}],[{'PU1': True}],[{'PU1': True}]
5,Megakaryocyte,Granulocyte,"[{'PU1': True, 'Gfi1': True, 'CEBPa': True, 'S...","[{'CEBPa': True, 'cJun': False}, {'Gfi1': True...","[{'CEBPa': True, 'cJun': False}, {'CEBPa': Tru..."
6,Monocyte,Erythrocyte,"[{'EKLF': True, 'GATA1': True, 'PU1': False}]","[{'PU1': False, 'Fli1': False, 'GATA2': True},...","[{'PU1': False, 'Fli1': False, 'GATA1': True},..."
7,Monocyte,Megakaryocyte,"[{'PU1': False, 'Fli1': True, 'GATA1': True}]","[{'PU1': False, 'Fli1': True}]","[{'Fli1': True, 'PU1': False}]"
8,Monocyte,Granulocyte,"[{'Gfi1': True, 'CEBPa': True, 'EgrNab': False}]","[{'cJun': False, 'CEBPa': True}, {'CEBPa': Tru...","[{'CEBPa': True, 'cJun': False}, {'CEBPa': Tru..."
9,Granulocyte,Erythrocyte,"[{'GATA1': True, 'CEBPa': False, 'EKLF': True,...","[{'GATA2': True, 'PU1': False, 'Fli1': False},...","[{'PU1': False, 'Fli1': False, 'GATA1': True},..."


In [18]:
for _, r in df.iterrows():
    print(r['source'], "->", r["target"], ':', r['permanent'])

Erythrocyte -> Megakaryocyte : [{'Fli1': True}, {'EKLF': False}]
Erythrocyte -> Monocyte : [{'PU1': True}]
Erythrocyte -> Granulocyte : [{'CEBPa': True, 'cJun': False}, {'Gfi1': True, 'CEBPa': True}, {'CEBPa': True, 'EgrNab': False}]
Megakaryocyte -> Erythrocyte : [{'Fli1': False}, {'EKLF': True}]
Megakaryocyte -> Monocyte : [{'PU1': True}]
Megakaryocyte -> Granulocyte : [{'CEBPa': True, 'cJun': False}, {'CEBPa': True, 'Gfi1': True}, {'EgrNab': False, 'CEBPa': True}]
Monocyte -> Erythrocyte : [{'PU1': False, 'Fli1': False, 'GATA1': True}, {'GATA1': True, 'EKLF': True, 'PU1': False}]
Monocyte -> Megakaryocyte : [{'Fli1': True, 'PU1': False}]
Monocyte -> Granulocyte : [{'CEBPa': True, 'cJun': False}, {'CEBPa': True, 'Gfi1': True}, {'CEBPa': True, 'EgrNab': False}]
Granulocyte -> Erythrocyte : [{'PU1': False, 'Fli1': False, 'GATA1': True}, {'GATA1': True, 'EKLF': True, 'PU1': False}]
Granulocyte -> Megakaryocyte : [{'PU1': False, 'Fli1': True}]
Granulocyte -> Monocyte : [{'CEBPa': False}]

From this data, we can (manually) derive a following source-target diagram, showing all minimal source-target controls.

![Source_target_controls](myeloid_source_target.png)

In [19]:
df['min_onestep_size'] = df['onestep'].apply(lambda x: len(x[0]))
df['min_onestep_options'] = df['onestep'].apply(lambda x: len(x))
df['min_temporary_size'] = df['temporary'].apply(lambda x: len(x[0]))
df['min_temporary_options'] = df['temporary'].apply(lambda x: len(x))
df['min_permanent_size'] = df['permanent'].apply(lambda x: len(x[0]))
df['min_permanent_options'] = df['permanent'].apply(lambda x: len(x))

In [20]:
df.filter(regex='^(source|target|min_)')

,source,target,min_onestep_size,min_onestep_options,min_temporary_size,min_temporary_options,min_permanent_size,min_permanent_options
0,Erythrocyte,Megakaryocyte,2,1,1,2,1,2
1,Erythrocyte,Monocyte,1,1,1,1,1,1
2,Erythrocyte,Granulocyte,3,1,2,3,2,3
3,Megakaryocyte,Erythrocyte,2,1,1,2,1,2
4,Megakaryocyte,Monocyte,1,1,1,1,1,1
5,Megakaryocyte,Granulocyte,4,3,2,3,2,3
6,Monocyte,Erythrocyte,3,1,3,6,3,2
7,Monocyte,Megakaryocyte,3,1,2,1,2,1
8,Monocyte,Granulocyte,3,1,2,4,2,3
9,Granulocyte,Erythrocyte,4,1,3,6,3,2


In [21]:
# In a similar manner, we can compute phenotype control, even though this time, only target attractor
# (or phenotype, but in this case we consider attractor to be phenotype) is known.

df['phenotype'] = df['target'].apply(lambda t: pick_result(ba.Control.phenotype_permanent(
    graph=pstg,
    phenotype=all_atts[t],
    oscillation_type="forbidden",
    size_limit=10,
    stop_when_found=True
)))

In [22]:
df['min_phenotype_size'] = df['phenotype'].apply(lambda x: len(x[0]))
df['min_phenotype_options'] = df['phenotype'].apply(lambda x: len(x))

In [23]:
df.filter(regex='^(source|target|min_)')

,source,target,min_onestep_size,min_onestep_options,min_temporary_size,min_temporary_options,min_permanent_size,min_permanent_options,min_phenotype_size,min_phenotype_options
0,Erythrocyte,Megakaryocyte,2,1,1,2,1,2,2,1
1,Erythrocyte,Monocyte,1,1,1,1,1,1,2,1
2,Erythrocyte,Granulocyte,3,1,2,3,2,3,2,3
3,Megakaryocyte,Erythrocyte,2,1,1,2,1,2,3,2
4,Megakaryocyte,Monocyte,1,1,1,1,1,1,2,1
5,Megakaryocyte,Granulocyte,4,3,2,3,2,3,2,3
6,Monocyte,Erythrocyte,3,1,3,6,3,2,3,2
7,Monocyte,Megakaryocyte,3,1,2,1,2,1,2,1
8,Monocyte,Granulocyte,3,1,2,4,2,3,2,3
9,Granulocyte,Erythrocyte,4,1,3,6,3,2,3,2


We can use the phenotype control results to compare whether they are in align with phenotype branching observed in the source of the model

In [24]:
for a in all_atts.keys():
    pd_select = df.loc[df['target'] == a, 'phenotype'].iloc[0]
    print(a, pd_select)

Erythrocyte [{'GATA1': True, 'Fli1': False, 'PU1': False}, {'EKLF': True, 'GATA1': True, 'PU1': False}]
Megakaryocyte [{'PU1': False, 'Fli1': True}]
Monocyte [{'CEBPa': False, 'PU1': True}]
Granulocyte [{'cJun': False, 'CEBPa': True}, {'Gfi1': True, 'CEBPa': True}, {'CEBPa': True, 'EgrNab': False}]


We can also compare the results with the original study

![image](myeloid_phenotype.png)
![image](phenotypes.png)


We can see that the results are consistent, because:

- Phenotype control is very similar to the original study. Since study does not assume all initial states, the control is slightly different and there are some extra variables to control (i.e. min phenotype controls with size greater than 2) but they are not in contradiction with the results. 
- There are some extra possible ways for the phenotype control, esp. negation of the last branch
- Phenotype control is for most of the cases is a super set of the source-targets control. No source-target control is in a contradiction with the phenotype control.
    - This shows that if we know the source and specific target in advance, we can further minimize the size of the control

## Everything below is just WIP

Now let's imagine, that some functions of the original model are unknown

![image](unknowns.png)

In [25]:
# Load the model and create a Perturbation Graph

model_unknown = ba.BooleanNetwork.from_file('myeloid_3unknown.aeon')
pstg_unknown = ba.AsynchronousPerturbationGraph(model_unknown)

In [26]:
# From the original graph, we can see how many colours there are (perturbed contains extra colors representing perturbations)

ba.AsynchronousGraph(model_unknown).mk_unit_colors()
CARDINALITY = ba.AsynchronousGraph(model_unknown).mk_unit_colors().cardinality()
CARDINALITY

2963088

In [31]:
# Find attractors representing the four possible phenotypes

attractors = ba.Attractors.attractors(pstg_unknown)
attractors = [ a.vertices() for a in attractors ]

erythrocyte = pstg_unknown.mk_subspace_vertices({ "EKLF": True })
erythrocyte_att = [ a for a in attractors if not a.intersect(erythrocyte).is_empty() ][0]
print('erythrocyte attractor:',erythrocyte_att.cardinality())

megakaryocyte = pstg_unknown.mk_subspace_vertices({ "Fli1": True })
megakaryocyte_att = [ a for a in attractors if not a.intersect(megakaryocyte).is_empty() ][0]
print('megakaryocyte attractor:', megakaryocyte_att.cardinality())

monocyte = pstg_unknown.mk_subspace_vertices({ "cJun": True })
monocyte_att = [ a for a in attractors if not a.intersect(monocyte).is_empty() ][0]
print('monocyte attractor:', monocyte_att.cardinality())


granulocyte = pstg_unknown.mk_subspace_vertices({ "Gfi1": True })
granulocyte_att = [ a for a in attractors if not a.intersect(granulocyte).is_empty() ][0]
print('granulocyte attractor:', granulocyte_att.cardinality())

[['erythrocyte', 'granulocyte'], ['erythrocyte'], ['granulocyte', 'megakaryocyte'], ['megakaryocyte']] ColorSet(cardinality=4221, symbolic_size=213)
[['granulocyte'], ['monocyte'], []] ColorSet(cardinality=396, symbolic_size=141)
[['erythrocyte'], ['granulocyte'], ['megakaryocyte']] ColorSet(cardinality=396, symbolic_size=141)
[['granulocyte', 'megakaryocyte'], ['megakaryocyte', 'monocyte']] ColorSet(cardinality=58086, symbolic_size=408)
[['erythrocyte'], ['granulocyte'], ['megakaryocyte'], ['monocyte']] ColorSet(cardinality=1281078, symbolic_size=858)
[['erythrocyte', 'granulocyte'], ['erythrocyte'], ['granulocyte', 'megakaryocyte'], ['granulocyte'], ['megakaryocyte'], ['monocyte']] ColorSet(cardinality=78885, symbolic_size=453)
[['granulocyte'], ['monocyte']] ColorSet(cardinality=58086, symbolic_size=408)
[['megakaryocyte']] ColorSet(cardinality=53865, symbolic_size=279)
[['erythrocyte'], ['megakaryocyte']] ColorSet(cardinality=53865, symbolic_size=279)
[['erythrocyte'], ['granulocyt

In [32]:
# TODO: There were 6 single-state attractors in the original model, why there is just one in the version of the model with three unknown variables?
attractors = ba.Attractors.attractors(pstg)
attractors = [ a.vertices().cardinality() for a in attractors ]
print(attractors)

attractors = ba.Attractors.attractors(pstg_unknown)
attractors = [ a.vertices().cardinality() for a in attractors ]
print(attractors)

In [38]:
# erythrocyte = pstg_unknown.mk_subspace_vertices({'Gfi1': False, 'EKLF': True, 'FOG1': True, 'GATA2': False, 'PU1': False, 'GATA1': True, 'SCL': True, 'cJun': False, 'Fli1': False, 'CEBPa': False, 'EgrNab': False})
# 
# megakaryocyte = pstg_unknown.mk_subspace_vertices(({'CEBPa': False,'EKLF': False,'EgrNab': False,'FOG1': True,'Fli1': True,'GATA1': True,'GATA2': False,'Gfi1': False,'PU1': False,'SCL': True,'cJun': False}))
# 
# monocyte = pstg_unknown.mk_subspace_vertices({'CEBPa': False,'EKLF': False,'EgrNab': True,'FOG1': False,'Fli1': False,'GATA1': False,'GATA2': False,'Gfi1': False,'PU1': True,'SCL': False,'cJun': True})
# 
# granulocyte = pstg_unknown.mk_subspace_vertices({'CEBPa': True,'EKLF': False,'EgrNab': False,'FOG1': False,'Fli1': False,'GATA1': False,'GATA2': False,'Gfi1': True,'PU1': True,'SCL': False,'cJun': False})
# 
# all_atts = {
#     "Erythrocyte": erythrocyte_att,
#     "Megakaryocyte": megakaryocyte_att,
#     "Monocyte": monocyte_att,
#     "Granulocyte": granulocyte_att
# }
# 
# # Generate al pair possibilities for computing the source-target control
# all_att_pairs = [(s,t) for s in all_atts.keys() for t in all_atts.keys() if s != t]

Erythrocyte -> Megakaryocyte : [{'Fli1': True}, {'EKLF': False}] : [0.848605, 0.848605]
Erythrocyte -> Monocyte : [{'PU1': True}] : [0.019736]
Erythrocyte -> Granulocyte : [{'CEBPa': True, 'cJun': False}, {'Gfi1': True, 'CEBPa': True}, {'CEBPa': True, 'EgrNab': False}] : [0.122306, 0.122306, 0.122306]
Megakaryocyte -> Erythrocyte : [{'Fli1': False}, {'EKLF': True}] : [0.848605, 0.424436]
Megakaryocyte -> Monocyte : [{'PU1': True}] : [0.019736]
Megakaryocyte -> Granulocyte : [{'CEBPa': True, 'cJun': False}, {'CEBPa': True, 'Gfi1': True}, {'EgrNab': False, 'CEBPa': True}] : [0.122306, 0.122306, 0.122306]
Monocyte -> Erythrocyte : [{'PU1': False, 'Fli1': False, 'GATA1': True}, {'GATA1': True, 'EKLF': True, 'PU1': False}] : [0.642197, 0.321098]
Monocyte -> Megakaryocyte : [{'Fli1': True, 'PU1': False}] : [0.476685]
Monocyte -> Granulocyte : [{'CEBPa': True, 'cJun': False}, {'CEBPa': True, 'Gfi1': True}, {'CEBPa': True, 'EgrNab': False}] : [0.251692, 0.251692, 0.251692]
Granulocyte -> Eryth

In [44]:
df

[['erythrocyte', 'granulocyte'], ['erythrocyte'], ['granulocyte', 'megakaryocyte'], ['granulocyte'], ['megakaryocyte'], ['monocyte']] ColorSet(cardinality=161556480, symbolic_size=453)
[['erythrocyte'], ['granulocyte'], ['megakaryocyte']] ColorSet(cardinality=811008, symbolic_size=141)
[['erythrocyte'], ['granulocyte'], ['megakaryocyte'], ['monocyte'], []] ColorSet(cardinality=9455616, symbolic_size=165)
[['erythrocyte'], ['granulocyte'], ['megakaryocyte'], ['monocyte']] ColorSet(cardinality=2623647744, symbolic_size=858)
[['granulocyte', 'megakaryocyte'], ['megakaryocyte', 'monocyte'], ['megakaryocyte']] ColorSet(cardinality=2795470848, symbolic_size=675)
[['granulocyte', 'megakaryocyte'], ['megakaryocyte']] ColorSet(cardinality=9455616, symbolic_size=197)
[['granulocyte'], ['monocyte'], []] ColorSet(cardinality=811008, symbolic_size=141)
[['granulocyte'], ['monocyte']] ColorSet(cardinality=118960128, symbolic_size=408)
[['granulocyte', 'megakaryocyte'], ['megakaryocyte', 'monocyte']]

In [52]:
for _, r in df.iterrows():
    control = ba.Control.attractor_permanent(pstg_unknown, all_atts[r['source']], all_atts[r['target']])
    robustness = [control.perturbation_robustness(p) for p in r['permanent']]
    print(r['source'], "->", r["target"], ':', r['permanent'], ':', robustness)


[({'Fli1': False}, 0.039206), ({'EKLF': True}, 1.0)]


In [55]:
ba.LOG_LEVEL = ba.LOG_VERBOSE

[['erythrocyte'], ['granulocyte'], ['megakaryocyte'], ['monocyte'], []] ColorSet(cardinality=1165350912, symbolic_size=847)
[['megakaryocyte'], []] ColorSet(cardinality=1351913472, symbolic_size=526)
[['granulocyte'], ['monocyte']] ColorSet(cardinality=13547520, symbolic_size=194)
[['erythrocyte'], ['granulocyte'], ['megakaryocyte'], ['monocyte']] ColorSet(cardinality=278206464, symbolic_size=773)
[[]] ColorSet(cardinality=46227456, symbolic_size=258)
[['monocyte'], []] ColorSet(cardinality=73543680, symbolic_size=260)
[['granulocyte'], ['monocyte'], []] ColorSet(cardinality=59996160, symbolic_size=322)
[['megakaryocyte']] ColorSet(cardinality=118960128, symbolic_size=383)
[['granulocyte'], []] ColorSet(cardinality=46227456, symbolic_size=258)
[['megakaryocyte'], ['monocyte'], []] ColorSet(cardinality=1443557376, symbolic_size=425)
[['erythrocyte'], ['megakaryocyte'], []] ColorSet(cardinality=9455616, symbolic_size=197)
[['erythrocyte'], ['granulocyte'], ['megakaryocyte'], []] ColorSet

In [64]:
# An example of the phenotype control computation

# TODO: This will compute ad infinitum, removing phen_sym_results.select_by_size helps

phen_sym_results = ba.Control.phenotype_permanent(
    graph=pstg_unknown,
    phenotype=erythrocyte_att,
    oscillation_type="forbidden",
    size_limit=2,
    stop_when_found=True,
)

# Why there is a working phenotype control of 1? Probably attractor got matched to some other state?
print([r for r in phen_sym_results.select_by_size(size=2, up_to=True)])

[({'Fli1': False, 'PU1': False, 'GATA2': True}, 0.921052), ({'PU1': False, 'GATA1': True, 'Fli1': False}, 0.921052), ({'CEBPa': False, 'Fli1': False, 'PU1': False}, 0.921052), ({'Fli1': False, 'PU1': False}, 0.848337), ({'Fli1': False, 'cJun': False, 'PU1': False}, 0.848337), ({'cJun': True, 'PU1': False, 'Fli1': False}, 0.848337), ({'Fli1': False, 'PU1': False, 'SCL': True}, 0.848337), ({'Gfi1': False, 'PU1': False, 'Fli1': False}, 0.848337), ({'Fli1': False, 'PU1': False, 'EgrNab': False}, 0.848337), ({'PU1': False, 'EgrNab': True, 'Fli1': False}, 0.848337), ({'PU1': False, 'Fli1': False, 'EKLF': True}, 0.848337), ({'GATA1': True, 'GATA2': True, 'Fli1': False}, 0.671052), ({'CEBPa': False, 'Fli1': False, 'GATA2': True}, 0.5), ({'CEBPa': False, 'Fli1': False, 'GATA1': True}, 0.5), ({'SCL': True, 'GATA1': True, 'Fli1': False}, 0.460526), ({'Fli1': False, 'PU1': False, 'FOG1': False}, 0.460526), ({'FOG1': True, 'Fli1': False, 'PU1': False}, 0.460526), ({'GATA2': True, 'EKLF': True, 'PU1

In [65]:
# We can find out the robustness of previously computed perturbations
mega_phen = ba.Control.phenotype_permanent(
    graph=pstg_unknown,
    phenotype=megakaryocyte_att,
    oscillation_type="forbidden",
    size_limit=3,
    stop_when_found=False
)

[({'PU1': True, 'Gfi1': True}, 0.5), ({'PU1': True, 'cJun': False}, 0.460526), ({'PU1': True, 'EgrNab': False}, 0.460526), ({'GATA1': False, 'Gfi1': True}, 0.233379), ({'cJun': False, 'GATA1': False}, 0.230263), ({'EgrNab': False, 'GATA1': False}, 0.230263), ({'cJun': False, 'CEBPa': True}, 0.143044), ({'Gfi1': True, 'CEBPa': True}, 0.143044), ({'EgrNab': False, 'CEBPa': True}, 0.143044)]


In [66]:
p = { v: None for v in pstg_unknown.perturbable_network_variable_names() }
p["Fli1"] = True
p["PU1"] = False
mega_phen.perturbation_robustness(p)

[({'PU1': False, 'Fli1': True}, 1.0), ({'EKLF': False, 'PU1': False}, 0.960526), ({'PU1': False}, 0.5), ({'cJun': False, 'PU1': False}, 0.5), ({'PU1': False, 'SCL': False}, 0.5), ({'SCL': True, 'PU1': False}, 0.5), ({'Gfi1': False, 'PU1': False}, 0.5), ({'PU1': False, 'Gfi1': True}, 0.5), ({'PU1': False, 'GATA2': False}, 0.5), ({'PU1': False, 'GATA2': True}, 0.5), ({'GATA1': True, 'cJun': False}, 0.5), ({'GATA1': True, 'PU1': False}, 0.5), ({'GATA1': True, 'Gfi1': True}, 0.5), ({'cJun': False, 'FOG1': True}, 0.5), ({'FOG1': True, 'PU1': False}, 0.5), ({'FOG1': True, 'Gfi1': True}, 0.5), ({'EgrNab': False, 'PU1': False}, 0.5), ({'EgrNab': True, 'PU1': False}, 0.5), ({'PU1': False, 'CEBPa': False}, 0.5), ({'PU1': False, 'CEBPa': True}, 0.5), ({'EKLF': True, 'PU1': False}, 0.460526), ({'GATA2': True, 'GATA1': True}, 0.355263), ({'Fli1': True, 'GATA1': True}, 0.289473), ({'EKLF': False, 'GATA1': True}, 0.289473), ({'CEBPa': False, 'Fli1': True}, 0.286088), ({'EKLF': False, 'CEBPa': False},

In [73]:
# We can find out the robustness of previously computed perturbations
mega_erythro = ba.Control.attractor_permanent(
    graph=pstg_unknown,
    source=megakaryocyte_att.pick_singleton(),
    target=erythrocyte_att.pick_singleton()
)

[({'PU1': True, 'Gfi1': False}, 0.072714), ({'Gfi1': False, 'GATA2': False}, 0.072714), ({'PU1': True, 'EgrNab': True}, 0.072714), ({'GATA2': False, 'EgrNab': True}, 0.072714), ({'Gfi1': False, 'CEBPa': True}, 0.036395), ({'EgrNab': True, 'CEBPa': True}, 0.036395), ({'GATA1': False, 'Gfi1': False}, 0.036357), ({'GATA1': False, 'EgrNab': True}, 0.036357), ({'Gfi1': False, 'SCL': False}, 0.006955), ({'SCL': False, 'EgrNab': True}, 0.006955), ({'Gfi1': False}, 0.004464), ({'EgrNab': True}, 0.004464), ({'cJun': True, 'Gfi1': False}, 0.004464), ({'Fli1': False, 'Gfi1': False}, 0.004464), ({'Fli1': True, 'Gfi1': False}, 0.004464), ({'cJun': False, 'EgrNab': True}, 0.004464), ({'cJun': True, 'EgrNab': True}, 0.004464), ({'EgrNab': True, 'Gfi1': False}, 0.004464), ({'Fli1': False, 'EgrNab': True}, 0.004464), ({'EgrNab': True, 'Fli1': True}, 0.004464), ({'EKLF': False, 'Gfi1': False}, 0.004464), ({'EKLF': False, 'EgrNab': True}, 0.004464), ({'Gfi1': False, 'EKLF': True}, 0.004464), ({'EgrNab': 

In [71]:
p = { v: None for v in pstg_unknown.perturbable_network_variable_names() }
p["EKLF"] = True
mega_erythro.perturbation_robustness(p)

[]


In [ ]:
results = mega_erythro.select_by_size(size=3, up_to=True).select_by_robustness(
    threshold=0.99,    
    result_limit=100
)
for (model, r, colors) in results:
    print(model.perturbed_named_dict(), r, colors)